## R script for OMOP GIS use cases

### Setup

In [1]:
library(DBI)

# make db connection
# for format of database file see db/env/db_conf.txt
db <- read.delim( '../../db/env/feta.txt', header=TRUE, sep=' ' )
con <- dbConnect(RPostgres::Postgres(),
                 dbname = toString(db$database[1]),
                 host = toString(db$host[1]),
                 port = 5432,
                 user = toString(db$user),
                 password = toString(db$pass))

# check the connection
dbListTables(con)

[1] "spatial_ref_sys"      "geography_columns"    "geometry_columns"    
 [4] "raster_columns"       "raster_overviews"     "hazard_point"        
 [7] "data_source"          "dd_type"              "dd_att_unit"         
[10] "dd_att_name"          "hz_att_category"      "locations"           
[13] "utm_grid"             "hz_att_unit"          "hz_att_name"         
[16] "hz_type"              "persons"              "care_site"           
[19] "dd_att_category"      "hz_attribute"         "location_history"    
[22] "site_history"         "dd_attribute"         "demographic_division"
[25] "sites"

### Examples

### 1 - get all patients within a particular census tract
WGS84 - ESPG 4326 (latitude and longitude)

In [ ]:
sql <- "SELECT row_number() over () AS _uid_,* FROM (
          with census_tract as (select geom from demographic_division where code = '12086009100') 
          select PS.*,L.geom from persons PS 
          Inner Join location_history LH on PS.person_id = LH.entity_id 
          Inner Join locations L on L.location_id = LH.location_id 
          join census_tract on ST_Within(L.geom, census_tract.geom) 
        ) _subq1"
res <- dbSendQuery(con,sql)
dbFetch(res)
dbClearResult(res)

### 2 - get all patients within a census tract with annual income below $15,000

**Use Case:** Patients with demographic criteria X that live within area Y, for duration Z  
**Example:** Patients who are over 100 years old that live within the TMC hospital service area for over 5 years  
**Functionality Requirements:** 3a, 3b, 4b  

**note:** this does not have the 'duration' component 

WGS84 - ESPG 4326 (latitude and longitude)

In [ ]:
sql <- "SELECT row_number() over () AS _uid_,* FROM (
          with census_tracts as (
            select DD.code, ATT.value, DD.geom from dd_attribute ATT
            Inner Join demographic_division DD on ATT.demographic_division_id = DD.id
            where ATT.dd_name_id = 1 and ATT.value < 15000
          ) 
          select PS.*,L.geom from persons PS 
          Inner Join location_history LH on PS.person_id = LH.entity_id 
          Inner Join locations L on L.location_id = LH.location_id 
          join census_tracts on ST_Within(L.geom, census_tracts.geom) 
        ) _subq1"
res <- dbSendQuery(con,sql)
dbFetch(res)
dbClearResult(res)

### 3 - get all patients within 1km of a carcinogen emitter (TRI data)

**Use Case:** Patients who live within distance X of non-clinical feature Y  
**Example:** Patients who live within 5 miles of a paper mill  
**Functionality Requirements:** 3a, 4a, 6a/6b, 3e  

WGS84 - ESPG 4326 (latitude and longitude)

In [ ]:
sql <- "with buffers as (
            select HZP.name, ST_Buffer(ST_Transform(HZP.geom,32618),1000) as geom from hz_attribute ATT
            Inner Join hazard_point HZP on ATT.hz_point_id = HZP.id
            where ATT.hz_category_id = 1 or ATT.hz_category_id = 3
        ) 
        select PS.*,L.geom from persons PS 
        Inner Join location_history LH on PS.person_id = LH.entity_id 
        Inner Join locations L on L.location_id = LH.location_id 
        join buffers on ST_Within(ST_Transform(L.geom,32618), buffers.geom)"
res <- dbSendQuery(con,sql)
dbFetch(res)
dbClearResult(res)

UTM 18N - EPSG 32618 (meters) - UM schema

In [ ]:
sql <- "with buffers as (
            select HZP.name, ST_Buffer(HZP.local_geom,1000) as local_geom from hz_attribute ATT
            Inner Join hazard_point HZP on ATT.hz_point_id = HZP.id
            where ATT.hz_category_id = 1 or ATT.hz_category_id = 3
        ) 
        select PS.*,L.geom from persons PS 
        Inner Join location_history LH on PS.person_id = LH.entity_id 
        Inner Join locations L on L.location_id = LH.location_id 
        join buffers on ST_Within(L.local_geom, buffers.local_geom)"
res <- dbSendQuery(con,sql)
dbFetch(res)
dbClearResult(res)

UTM 18N - EPSG 32618 (meters) - ohdsi schema

In [ ]:
sql <- "with buffers as (
            select geo_florida_tri_2018.name, ST_Buffer(geo_florida_tri_2018.geom_local,1000) as geom_local from attr_florida_tri_2018 ATT
            Inner Join hazard_point HZP on ATT.geo_record_id = geo_florida_tri_2018.geo_record_id
            where ATT.concept_id = 507 and (ATT.value_as_concept_id = 1 or ATT.hz_category_id = 3
        ) 
        select PS.*,L.geom from persons PS 
        Inner Join location_history LH on PS.person_id = LH.entity_id 
        Inner Join locations L on L.location_id = LH.location_id 
        join buffers on ST_Within(L.local_geom, buffers.local_geom)"
res <- dbSendQuery(con,sql)
dbFetch(res)
dbClearResult(res)

### 4 - get all patients within census block that contains a carcinogen emitter (TRI data)

**Use Case:** Patients who live in an area that contains a non-clinical feature Y	
**Example:** Paper mill in same census tract as residence	
**Functionality Requirements:** 3a, 4a, 4b

UTM 18N - ESPG 32618 (meters)

In [ ]:
sql <- "SELECT row_number() OVER () AS _uid_,* FROM (
          WITH CG AS (
            SELECT HZP.name, HZP.local_geom FROM hz_attribute ATT
            INNER JOIN hazard_point HZP ON ATT.hz_point_id = HZP.id
            WHERE ATT.hz_category_id = 1 OR ATT.hz_category_id = 3
          ), 
          CT AS (
            SELECT DD.code, DD.local_geom FROM demographic_division DD
            JOIN CG ON ST_Contains(DD.local_geom,CG.local_geom)
          ) 
          SELECT PS.*,L.local_geom FROM persons PS 
          INNER JOIN location_history LH ON PS.person_id = LH.entity_id 
          INNER JOIN locations L ON L.location_id = LH.location_id 
          JOIN CT ON ST_Within(L.local_geom, CT.local_geom) 
        ) _subq1"
res <- dbSendQuery(con,sql)
dbFetch(res)
dbClearResult(res)

### 5 - get all patients within 1km of network care sites

**Use Case:** Given a list of care sites, patients who lived within Y miles on date range Z  	
**Example:** For a list of in-network care sites, which patients live within 10 miles  
**Functionality Requirements:** 3a, 3b, 4a, 6b

UTM 18N - EPSG 32618 (meters)

In [ ]:
sql <- "WITH CB AS (
            SELECT CS.care_site_name, ST_Buffer(CSL.local_geom,1000) AS local_geom FROM care_site CS
            INNER JOIN site_history SH ON SH.entity_id = CS.care_site_id
            INNER JOIN sites CSL ON SH.site_id = CSL.site_id
            WHERE CS.place_of_service_concept_id = 0
        ) 
        SELECT PS.*,L.local_geom FROM persons PS 
        INNER JOIN location_history LH ON PS.person_id = LH.entity_id 
        INNER JOIN locations L ON L.location_id = LH.location_id 
        JOIN CB ON ST_Within(L.local_geom, CB.local_geom)"
res <- dbSendQuery(con,sql)
dbFetch(res)
dbClearResult(res)

In [ ]:
sql <- "SELECT PS.*,L.local_geom FROM persons PS, care_site CS, sites CSL, location_history LH, locations L
        Where CS.place_of_service_concept_id = 0
        and CS.site_id = CSL.site_id
        and PS.person_id = LH.entity_id
        and L.location_id = LH.location_id
        and ST_Within(L.local_geom, ST_Buffer(CSL.local_geom, 1000))"
res <- dbSendQuery(con,sql)
dbFetch(res)
dbClearResult(res)

### 6 - get all patients who are more than 5 kilometers from a care site

**Use Case:** Patients who traveled over Y miles for primary care visit  	
**Example:** Patients who must travel greater than 50 miles to PCP
**Functionality Requirements:** 3a, 3b, 6a

UTM 18N - EPSG 32618 (meters)

In [ ]:
sql <- "WITH CB AS (
            SELECT CS.care_site_name, ST_Buffer(CSL.local_geom,5000) AS local_geom FROM care_site CS
            INNER JOIN site_history SH ON SH.entity_id = CS.care_site_id
            INNER JOIN sites CSL ON SH.site_id = CSL.site_id
            WHERE CS.place_of_service_concept_id = 0
        ) 
        SELECT persons.*,L.local_geom FROM persons 
        INNER JOIN location_history LH ON person_id = LH.entity_id 
        INNER JOIN locations L ON L.location_id = LH.location_id
        WHERE person_id NOT IN (
            SELECT PS.person_id FROM persons PS
            INNER JOIN location_history LH ON PS.person_id = LH.entity_id 
            INNER JOIN locations L ON L.location_id = LH.location_id 
            JOIN CB ON ST_Within(L.local_geom, CB.local_geom)
        )"
res <- dbSendQuery(con,sql)
dbFetch(res)
dbClearResult(res)